Import Libraries

In [1]:
import pandas as pd
import ast
import numpy as np
import seaborn as sns
import progressbar
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

Read source data

In [2]:
df_Eu = pd.read_csv(r'..\Data\dataset_qual_gua.csv')
df_def = pd.read_excel(r'..\data\Parameters with Labels.xlsx',sheet_name = 'Lables').transpose()
df_peso = pd.read_excel(r'..\data\Tabla densidades_pesos.xlsx')
# df_TDS_4563 = pd.read_excel(r'..\data\TDS Tables.xlsx',sheet_name='4563')
# df_TDS_4587 = pd.read_excel(r'..\data\TDS Tables.xlsx',sheet_name='4587')
# df_TDS_4582 = pd.read_excel(r'..\data\TDS Tables.xlsx',sheet_name='4582')
# df_TDS_4620 = pd.read_excel(r'..\data\TDS Tables.xlsx',sheet_name='4620')
# df_TDS_4695 = pd.read_excel(r'..\data\TDS Tables.xlsx',sheet_name='4695')
# df_TDS_4696 = pd.read_excel(r'..\data\TDS Tables.xlsx',sheet_name='4696')
# df_TDS_4697 = pd.read_excel(r'..\data\TDS Tables.xlsx',sheet_name='4697')

Data Cleaning

In [3]:
delete_column_list = ['Path_EM63','DATE','BLADEID_path_file','BladeID_QN_path','TIME','bl.index','bl.TS',
                       'bl.ProdOrder','qn.index','qn.PartNumber','bl.PartNumber','qn.TS','qn.Mold','qn.Cavity','qn.Cycleid','@15241.1']
df_Eu = df_Eu.drop(delete_column_list,axis = 1)
df_Eu = df_Eu.rename(columns={'PartitionKey': 'BLADE_ID', 'RowKey': 'Reference','Production_Timestamp':'TimeStamp',
                        'bl.Cycleid':'CYCLE_ID','bl.Mold':'Mold','bl.Cavity':'Mold_Cavity','qn.DefectMES':'Defect_MES',
                        'qn.DefectERP':'Defect_ERP','qn.ProdOrder':'Quality_ID'})
df_peso.columns = ['Mold','Reference','Description','Cycle_Time','Weight','Sprue','Others','Total','Density(g/cm3)','Type','Grain_Density(g/cm3)']

Adding Quality Status column to the dataset

In [4]:
l = list(df_Eu['Quality_ID'])
a = []
for i in range(len(l)):
    if str(l[i]) !=  'nan':
        a.append(str(int(df_Eu['Defect_MES'][i])))
    else:
        a.append('OK')
df_Eu['Status'] = a
df_Eu_data = df_Eu.iloc[:,15:]

Creating Euromap code definition Dictionary

In [5]:
Euromap_Definition = {}
df_def.columns = df_def.iloc[0]
df_def = df_def[1:]
Euromap_Definition = df_def.to_dict()

In [6]:
list_df_filter = list(df_Eu.iloc[:,10:].columns)[0:-1]
list_df_Euromap_def = list(Euromap_Definition.keys())
list_code_missing = []
for code in list_df_filter:
    if code not in list_df_Euromap_def:
        list_code_missing.append(code)
print('There are total of {} missing EU63 parameter definition'.format(len(list_code_missing)))
list_code_missing

There are total of 14 missing EU63 parameter definition


['@0020',
 '@0021',
 '@0022',
 '@3731',
 '@4110',
 '@4111',
 '@4112',
 '@4113',
 '@4114',
 '@6111',
 '@6112',
 '@6113',
 '@6114',
 '@6115']

============================================================================================================================================================


Tolerance analysis - Passive

In [ ]:
result_tol_df_columns = list(df_Eu.columns)
result_tol_df_columns.extend(['tol_limit_variables','tol_limit_count'])
result_tol_df = pd.DataFrame(columns=result_tol_df_columns)

In [ ]:
df_tol = df_Eu.iloc[:,:]
bar = progressbar.ProgressBar(maxval=df_tol.shape[0]).start()
for i, j in df_tol.iterrows():
    bar.update(i)
    tol_df = pd.read_excel(r'..\data\TDS Tables.xlsx',sheet_name=j['Mold'].split('-')[0]).fillna(0)
    list_columns = [str(i) for i in list(tol_df['PARAMETER (@)'])]
    for col in range(j.shape[0]):
        try:
            if df_tol.columns[col].split('@')[1] not in list_columns:
                result_tol_df.at[i,df_tol.columns[col]] = 0
            elif df_tol.columns[col].split('@')[1] in list_columns:
                parameter_index = list_columns.index(df_tol.columns[col].split('@')[1])
                parameter_temp = df_tol.columns[col]
                parameter_real_value = j[col]
                nominal_value = float(tol_df['NOMINAL VALUE'][parameter_index])
                max_value = nominal_value + float(tol_df['MAX TOLERANCE'][parameter_index])
                min_value = nominal_value - float(tol_df['MIN TOLERANCE'][parameter_index])
                try:
                    final_parameter_value = (parameter_real_value-min_value)/(max_value-min_value)
                except ZeroDivisionError:
                    final_parameter_value = 0
                result_tol_df.at[i,parameter_temp] = final_parameter_value
        except IndexError:
            result_tol_df.at[i,df_tol.columns[col]] = 0
    result_tol_df.at[i, 'Mold'] = j['Mold']

In [ ]:
result_tol_df = result_tol_df.iloc[:,15:]
result_tol_df['Mold'] = df_Eu['Mold']
result_tol_df['Status'] = df_Eu['Status']
result_tol_df['tol_limit_variables'] =  result_tol_df['tol_limit_variables'].astype('object')

In [ ]:
bar = progressbar.ProgressBar(maxval=result_tol_df.shape[0]).start()
for i, j in result_tol_df.iterrows():
    bar.update(i)
    out = list()
    for col in range(j.shape[0]-4):
        if float(j[col])>1 or float(j[col])<0:
            out.append(result_tol_df.columns[col])
    result_tol_df.at[i,'tol_limit_variables']=out
    result_tol_df['tol_limit_count'][i] = len(result_tol_df['tol_limit_variables'][i])

In [ ]:
result_tol_df.to_excel('result_tolerance.xlsx',index=True)

================================================================================================================================================

Tolerance plots

In [9]:
tol_result = pd.read_excel('..\data\\result_tolerance.xlsx')

In [10]:
fig = px.histogram(tol_result, x="tol_limit_count",nbins=100,title="Variable deviation count")
fig.show()

In [29]:
list_limit = []
df_tol_count = pd.DataFrame(columns=['Parameter','Count'])
for i in range(tol_result.shape[0]):
    l1 = ast.literal_eval(tol_result['tol_limit_variables'][i])
    for item in l1:
        list_limit.append(item)
unique_tol = list(set(list_limit))
for param in unique_tol:
    c1 = list_limit.count(param)
    df_tol_count = df_tol_count.append({'Parameter': param,'Count':c1},ignore_index=True)

fig = px.bar(df_tol_count, x="Parameter",y='Count',color_discrete_sequence=["#85660D"],title="There are {} deviating variables".format(len(unique_tol)))
fig.show()

Sample 16 parameter type analysis

In [16]:
tol_result_16 = tol_result[tol_result['tol_limit_count']==16].reset_index(drop=True)
list_limit_16 = []
df_tol_count_16 = pd.DataFrame(columns=['Parameter','Count','Description'])
for i in range(tol_result_16.shape[0]):
    l2 = ast.literal_eval(tol_result_16['tol_limit_variables'][i])
    for item in l2:
        list_limit_16.append(item)
unique_tol_16 = list(set(list_limit_16))

if len(unique_tol_16)==16:
    print('There are same set of parameters go out of range in all recorded events')
else:
    print('')
    for param in unique_tol_16:
        c2 = list_limit_16.count(param)
        try:
            df_tol_count_16 = df_tol_count_16.append({'Parameter': param,'Count':c2,'Description':Euromap_Definition[param]['Description']},ignore_index=True)
        except KeyError:
            df_tol_count_16 = df_tol_count_16.append({'Parameter': param,'Count':c2,'Description':param},ignore_index=True)
fig = px.bar(df_tol_count_16, x="Parameter",y="Count",text='Description',color_discrete_sequence=["#72B7B2"], title="There are {} parameters deviate out of range in '16 type' recorded events".format(len(unique_tol_16)))
fig.show()

Sample 2 parameter type analysis

In [20]:
tol_result_2 = tol_result[tol_result['tol_limit_count']==2].reset_index(drop=True)
list_limit_2 = []
df_tol_count_2 = pd.DataFrame(columns=['Parameter','Count','Description'])
for i in range(tol_result_2.shape[0]):
    l3 = ast.literal_eval(tol_result_2['tol_limit_variables'][i])
    for item in l3:
        list_limit_2.append(item)
unique_tol_2 = list(set(list_limit_2))

if len(unique_tol_2)==2:
    print('There are same set of parameters go out of range in all recorded events')
else:
    print('')
    for param in unique_tol_2:
        c3 = list_limit_2.count(param)
        try:
            df_tol_count_2 = df_tol_count_2.append({'Parameter': param,'Count':c3,'Description':Euromap_Definition[param]['Description']},ignore_index=True)
        except KeyError:
            df_tol_count_2 = df_tol_count_2.append({'Parameter': param,'Count':c3,'Description':param},ignore_index=True)

fig = px.bar(df_tol_count_2, x="Parameter",y="Count",color_discrete_sequence=["#E45756"],text = 'Description', title="There are {} parameters deviate out of range in '2 type' recorded events".format(len(unique_tol_2)))
fig.show()

In [ ]:
pressure_df = tol_result.iloc[:,115:125]
pressure_df.head(5)

NOK Analysis

In [25]:
NOK_data = tol_result[tol_result['Status']!='OK'].drop('Index',axis=1).reset_index(drop=True)
fig = px.histogram(NOK_data, x="tol_limit_count",nbins=100,title="Variable deviation count in NOK events")
fig.show()

In [28]:
list_limit_NOK = []
df_tol_count_NOK = pd.DataFrame(columns=['Parameter','Count'])
for i in range(NOK_data.shape[0]):
    l4 = ast.literal_eval(NOK_data['tol_limit_variables'][i])
    for item in l4:
        list_limit_NOK.append(item)
unique_tol_NOK = list(set(list_limit_NOK))
for param in unique_tol_NOK:
    c4 = list_limit_NOK.count(param)
    df_tol_count_NOK = df_tol_count_NOK.append({'Parameter': param,'Count':c4},ignore_index=True)

fig = px.bar(df_tol_count_NOK, x="Parameter",y='Count',color_discrete_sequence=["#85660D"],title="There are {} deviating variables in NOK".format(len(unique_tol_NOK)))
fig.show()

In [30]:
tol_result_16_NOK = NOK_data[NOK_data['tol_limit_count']==16].reset_index(drop=True)
list_limit_16_NOK = []
df_tol_count_16_NOK = pd.DataFrame(columns=['Parameter','Count','Description'])
for i in range(tol_result_16_NOK.shape[0]):
    l5 = ast.literal_eval(tol_result_16_NOK['tol_limit_variables'][i])
    for item in l5:
        list_limit_16_NOK.append(item)
unique_tol_16_NOK = list(set(list_limit_16_NOK))

if len(unique_tol_16_NOK)==16:
    print('There are same set of parameters go out of range in all recorded events')
else:
    for param in unique_tol_16_NOK:
        c5 = list_limit_16_NOK.count(param)
        try:
            df_tol_count_16_NOK = df_tol_count_16_NOK.append({'Parameter': param,'Count':c5,'Description':Euromap_Definition[param]['Description']},ignore_index=True)
        except KeyError:
            df_tol_count_16_NOK = df_tol_count_16_NOK.append({'Parameter': param,'Count':c5,'Description':param},ignore_index=True)
fig = px.bar(df_tol_count_16_NOK, x="Parameter",y="Count",text='Description',color_discrete_sequence=["#72B7B2"], title="There are {} parameters deviate out of range in '16 type' recorded events".format(len(unique_tol_16)))
fig.show()

In [51]:
compare_df = pd.DataFrame(columns=list(NOK_data.columns))
compare_df = compare_df.append(NOK_data[NOK_data['tol_limit_count']==16].iloc[:1,:])
compare_df = compare_df.append(tol_result[tol_result['tol_limit_count']==16].iloc[:1,:])
print(compare_df['tol_limit_variables'].iloc[0])
print(compare_df['tol_limit_variables'].iloc[1])

['@15218', '@24014', '@6101', '@6102', '@6103', '@6104', '@6105', '@6106', '@6107', '@6108', '@6109', '@6110', '@6111', '@6112', '@6113', '@6114']
['@15218', '@24014', '@6101', '@6102', '@6103', '@6104', '@6105', '@6106', '@6107', '@6108', '@6109', '@6110', '@6111', '@6112', '@6113', '@6114']


Feature Analysis

In [ ]:
one_hot_encoded_data = pd.get_dummies(df_Eu, columns = ['Status'])

# Random forest one hot encoding  
x = one_hot_encoded_data.iloc[:,15:-10]
y = one_hot_encoded_data.iloc[:,235:]

#SVM no one hot encoding
# x = df_Eu.iloc[:,15:-1]
# y = df_Eu['Status']

cor_matrix = x.corr()
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))

In [ ]:
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]
to_drop_column = []
for item in to_drop:
    if item not in list_code_missing:
        to_drop_column.append(Euromap_Definition[item]['Description'])
    else:
        to_drop_column.append(item)
to_drop_column

Classification

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
y_train.shape, y_test.shape

Random Forest

In [ ]:
rf = RandomForestClassifier(random_state=42, n_jobs=-1, max_depth=5,
                                       n_estimators=100, oob_score=True)
rf.fit(X_train, y_train)
rf.oob_score_

In [ ]:
rf_2 = RandomForestClassifier(random_state=42, n_jobs=-1)
params = {
    'max_depth': [2,3,5,10,20],
    'min_samples_leaf': [5,10,20,50,100,200],
    'n_estimators': [10,25,30,50,100,200]
}

In [ ]:
grid_search = GridSearchCV(estimator=rf_2,
                           cv = 4,
                           param_grid=params,
                           n_jobs=-1, verbose=1, scoring="accuracy")
grid_search.fit(X_train, y_train)
print("Optimal hyperparameter combination:", grid_search.best_params_)
print()
print("Mean cross-validated training accuracy score:",
      grid_search.best_score_)
grid_search.best_estimator_.fit(X_train, y_train)

In [ ]:
y_pred = grid_search.best_estimator_.predict(X_test) # Predictions
y_true = y_test.to_numpy() # True values

print("Test accuracy:", np.round(accuracy_score(y_true, y_pred), 2))
cf_matrix = confusion_matrix(y_true.argmax(axis=1), y_pred.argmax(axis=1),labels= list(set(y_true.argmax(axis=1))))
cf_matrix

In [ ]:
label_dict = {'0':'MES_3','1':'MES_8','2':'MES_10','3':'MES_14','4':'MES_19','5':'MES_20','6':'MES_21','7':'MES_112','8':'MES_118','9':'OK'}

In [ ]:
def get_value_list(a,dict = label_dict):
    dummy_list = []
    for item in a:
        dummy_list.append(dict[str(item)])   
    return dummy_list

In [ ]:
plt.figure(figsize=(15,7))
ax= plt.subplot()
sns.heatmap(cf_matrix, annot=True, cmap='Blues',annot_kws={"size": 15},ax=ax)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix')
ax.xaxis.set_ticklabels(get_value_list(list(set(y_true.argmax(axis=1))))) 
ax.yaxis.set_ticklabels(get_value_list(list(set(y_true.argmax(axis=1)))))

Support Vector Machine

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import learning_curve
svm = SVC(kernel='rbf', C=1, gamma='scale')
svm.fit(X_train,y_train)

In [ ]:
train_sizes, train_scores, test_scores = learning_curve(svm,X_train,y_train, cv=10)

In [ ]:
plt.plot(train_sizes,np.mean(train_scores,axis=1))
plt.plot(train_sizes,np.mean(test_scores,axis=1), label = 'Validation error')

In [ ]:
svm_pred = svm.predict(X_test)
svm_true = y_test.to_numpy() # True values

print("Test accuracy:", np.round(accuracy_score(svm_true, svm_pred), 2))
cf2_matrix = confusion_matrix(svm_true,svm_pred)
cf2_matrix